## Import all modules

In [153]:
from Bio import Seq
import urllib
from Bio import Entrez
#also set email address needed for using entrez
Entrez.email = 'jennala1487@gmail.com'
from Bio import SeqIO
import statistics as stat
from Bio.Seq import translate
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
from itertools import chain

## Change to proper working directory- used for reading and writing all files

In [112]:
cd /mnt/c/Users/jenna/OneDrive/Desktop/Rosalind

/mnt/c/Users/jenna/OneDrive/Desktop/Rosalind


## Problem 1 (INI): Count the number of each nt in a string

In [137]:
#read in string
filepath="rosalind_ini.txt"
seq = open(filepath, 'r').read().replace('\n', '')
    
#Print out the number of times each occurs
counts=[seq.count("A"), seq.count("C"), seq.count("G"), seq.count("T")]
print("Occurences of A/C/G/T: " + " ".join(map(str, counts)))


Occurences of A/C/G/T: 229 185 214 229


## Problem 2 (DPBR): Given a protein's ID, retrieve a list of biological processes it's involved in from UniProt

In [139]:
#Read in id
id=open("rosalind_dpbr.txt").read()

#get url for id
url="http://www.uniprot.org/uniprot/"+ id + ".txt"

#open up url
content=urllib.request.urlopen(url) 

#iterate through each line, saving lines with "GO", then split up to print just the process names
print("GO terms for " + id + ": ")
for line in content: 
    line=str(line)
    if "GO" in line:
        groups=line.split(";")
        group2=";".join(groups[2:3])
        print(group2.split(":")[1])

GO terms for Q5SLP9: 
single-stranded DNA binding
DNA recombination
DNA repair
DNA replication


## Problem 3 (GBK): given a genus name and a range of dates, return the number of entries in Nucleotide GenBank

In [120]:
#read in search terms
filepath="rosalind_gbk.txt"
searchList=open(filepath,'r').read().split("\n")

#assign to name. Assumes in format of genus \n start date \n end date
genus=searchList[0]
firstdate=searchList[1]
seconddate=searchList[2]

#get query to use
query= f'({genus}[Organism]) AND("{firstdate}"[Publication Date]: "{seconddate}"[Publication Date])'

#get the handle for the given organism
handle = Entrez.esearch(db="nucleotide", term=query)

#read record of the handle, print out the count
record = Entrez.read(handle)
print("Number of records for " + genus + " between " + firstdate + " and " + seconddate + ": " + record["Count"])

Number of records for Scarabaeus between 2004/03/31 and 2012/09/30: 989


## Problem 4 (FRMT): given list of genbank gene IDs, return the fasta file of the gene with the shortest header

In [121]:
#read in IDs
filepath="rosalind_frmt.txt"
IDs=open(filepath).read().replace("\n", "").split(" ")

In [123]:
#get the handle for the given IDs, returning fasta data
handle = Entrez.efetch(db="nucleotide", id=IDs, rettype="fasta")

#use SeqIO to read in record as fasta format
records = list (SeqIO.parse(handle, "fasta")) 

#get index of fasta with shortest description
lengths=[]

for record in records:
    lengths.append(len(record.seq))
    
minIndex=lengths.index(min(lengths))

#Print out this fasta record
print("Shortest fasta sequence of given gene IDs:")
print(records[minIndex].format("fasta"))
    

Shortest fasta sequence of given gene IDs:
>JX295575.1 Anopheles sinensis prophenoloxidase 1 (PPO-1) mRNA, partial cds
AACCTGCATCATTGGCATTGGCATCTTGTGTACCCGGGCGAAGGGCCCGATCGTGTCGTC
AACAAGGATCGTCGTGGAGAGTTGTTCTACTACATGCACCAGCAGCTGATCGCTCGCTAC
AACGTCGATCGCTTCTGCAACCGTTTGGCGCGGGTGCGTCCACTGACGAATCTGCGTGAG
CCTCTTCCGGAGGGATACTTCCCGAAACTCATCCGAAGCTTCACCAACCGTGCCTTCCCT
GCCCGACCTCAGAACCATATGTTGAGGGATTTGAATCGCATTGAGGACGATGTGGTACTC
TCGATCAGTGATCTGGAACGCTGGGGAAGCCGCATTGCCGAGAGCATTGATGGTGGATAC
GTGGTGGGCCCCGGTGGTGCACGTACTCCTCTGGATGAACAAACGGGTATCGACGTGCTG
GGCAACATCATGGAACCGTCGGCACTGTCGGTGAACCCGCAATTCTATGGAAACTACCAT
GGCCATATGCACAATCTCATCGCGTTCAGTCACGATCCTGAGAACCGCTTCCTGGAGGGG
TACGGTGTGGTGGGCGAGTTCCAGACGGCCATGCGTGACCCTACGTTCTACCGCTGG



## Problem 5 (TFSQ): given a file with fastqs, convert to fasta

In [132]:
#get path to string
filepath="rosalind_tfsq.txt"
fastafilepath=filepath.replace(".txt", ".fasta.txt")

#write out to fasta
SeqIO.convert(filepath,'fastq','output.fasta.txt','fasta')

print("fastq file \"" + filepath + "\" written to fasta file \"" + fastafilepath + "\"")

fastq file "rosalind_tfsq.txt" written to fasta file "rosalind_tfsq.fasta.txt"


## Problem 6 (PHRE): given a file with fastqs + quality threshold, print the number of fastqs that pass that threshold

In [135]:
#read in fastq as normal text, extract threshold
file="rosalind_phre.txt"
fileSplit=open(file).read().split("\n")
thresh=int(fileSplit[0])

#remove first line (threshold) from fastq, then write back out as same file
with open(file, 'r') as fin:
    data = fin.read().splitlines(True)
with open(file, 'w') as fout:
    fout.writelines(data[1:])

In [136]:
#Initialize count, then count the number of fastqs with a mean quality score less than the given threshold
count=0
for record in list(SeqIO.parse(file, "fastq")):
    avgQ=stat.mean(record.letter_annotations["phred_quality"])
    if avgQ<thresh:
        count=count+1
print(str(count) + " fastqs have a mean quality score less than " + str(thresh))        


29 fastqs have a mean quality score less than 20


## Problem 7 (BPHR): given a file with fastqs + quality threshold + % of bases that need to meet that theshold, print the number of fastqs that pass that threshold

In [143]:
#read in fastq as normal text, extract quality threshold and percentage threshold
file="rosalind_bphr.txt"
fileSplit=open(file).read().split("\n")

ScoreThresh=int(fileSplit[0].split(" ")[0])

#remove first line (threshold) from fastq, then write back out as same file
with open(file, 'r') as fin:
    data = fin.read().splitlines(True)
with open(file, 'w') as fout:
    fout.writelines(data[1:])

In [144]:
#Initialize count list
positions=[]
nRecords=0
LowScores=[]

#For every position across all fastqs, get the score. Add all scores at that position together
for record in list(SeqIO.parse(file, "fastq")):
    nRecords+=1
    q=record.letter_annotations["phred_quality"]
    for score in range(0,len(q)):
        try:
            positions[score]+= q[score]
        except:
            positions.append(q[score])

#for each position, get the mean score across all fastqs, test if greater than given threshold
for position in positions:
    MeanQual=position/nRecords
    if MeanQual<ScoreThresh:
        LowScores.append(MeanQual)
        
print(str(len(LowScores)) + " positions have a mean quality below " + str(ScoreThresh))        
    
   

7 positions have a mean quality below 18


## Problem 8 (BFIL): given a file with fastqs + quality threshold, trim bases from the beginning and end that don't meet that quality threshold

In [146]:
#read in fastq as normal text, extract quality threshold and percentage threshold
file="rosalind_bfil.txt"
fileSplit=open(file).read().split("\n")

ScoreThresh=int(fileSplit[0].split(" ")[0])

#remove first line (threshold) from fastq, then write back out as same file
with open(file, 'r') as fin:
    data = fin.read().splitlines(True)
with open(file, 'w') as fout:
    fout.writelines(data[1:])

In [157]:
#Initialize count list + file to write out
nRecords=0
myfile = open('TrimmedFastqs.txt', 'w')

#get list of scores for all records
for record in list(SeqIO.parse(file, "fastq")):
    nRecords+=1
    q=record.letter_annotations["phred_quality"]
    
    #for each record, get positions from START that are less than threshold. Break once you hit a high qual position
    lowQualStart=[]
    for i in range(len(q)):
        qual=q[i]
        if qual<ScoreThresh:
            lowQualStart.append(i)
        else:
            break
    
    #for each record, get positions from END that are less than threshold. Break once you hit a high qual position
    lowQualEnd=[]
    for i in reversed(range(len(q))):
        qual=q[i]
        if qual<ScoreThresh:
            lowQualEnd.append(i)
        else:
            break
    
    #bind low qual start/end indexes together, remove from seqs
    trim=[lowQualStart, lowQualEnd]
    trim=list(chain.from_iterable(trim))
    remove=list(set(list(range(0,len(q))))-set(trim))
    record_removed=record[min(remove):(max(remove)+1)]
    myfile.write(record_removed.format("fastq"))
    
myfile.close()    
       
print("Bases with quality less than " + str(ScoreThresh) + " trimmed from ends of " + str(nRecords) + " sequences and written to TrimmedFastqs.txt")   

Bases with quality less than 21 trimmed from ends of 95 sequences and written to TrimmedFastqs.txt


## Problem 9: for a file containing two genbank IDs, get alignment score using Needle for fasta sequences

In [164]:
#Read in IDs, export fasta files
filepath="rosalind_need.txt"
IDs=open(filepath).read().replace("\n", "").split(" ")

#get the handle for the given IDs, returning fasta data
handle = Entrez.efetch(db="nucleotide", id=IDs, rettype="fasta")

#use SeqIO to read in record as fasta format
records = list (SeqIO.parse(handle, "fasta")) 

#write out fasta to file
for record in records:
    filename=record.id + ".txt"
    myfile=open(filename, "w")
    myfile.write(record.format("fasta"))
    myfile.close()    

print("fasta files for " + IDs[0] + " and " + IDs[1] + " exported")
print("")
print("To get alignment score, run:")
print("needle " + IDs[0] + ".txt " + IDs[1] + ".txt -endweight -endopen 10 -endextend 1")
#Execute needle at command line: needle file1.txt file2.txt -endweight -endopen 10 -endextend 1    

fasta files for NM_001194889.1 and JX475045.1 exported

To get alignment score, run:
needle NM_001194889.1.txt JX475045.1.txt -endweight -endopen 10 -endextend 1


## Problem 10: for an input DNA and aa sequence it translates to, print the codon table (1=standard, other options are 2-15) that was used to translate

In [165]:
#Read in DNA and aa sequence
filepath="rosalind_ptra.txt"
both=open(filepath).read().split("\n")

In [169]:
#Cycle through all possible codon tables- 1-15, with at least 7-8 now deleted. Check which (may be multiple) would result in the given aa sequence
for i in range(1,16):
    try:
        translated=translate(both[0], table =i, stop_symbol="")
        if translated == both[1]:
            print("Codon table " + str(i) + " was used to translate the DNA sequence")
    except:
        print("")



Codon table 12 was used to translate the DNA sequence


## Problem 11: for a list of DNA sequence, print how many match their own reverse complement

In [170]:
#read in the dna sequences
filepath="rosalind_rvco.txt"

In [171]:
#initialize count variable
count=0

#loop through all entries, check if their RC matches the forward seq
for record in list(SeqIO.parse(filepath, "fasta")):
    seq=record.seq
    rc=seq.reverse_complement()
    if seq == rc:
        count+=1
print(str(count) + " sequences match their reverse complement")  

3 sequences match their reverse complement


## Problem 12: For a given DNA string, get the longest open reading frame (all frames, forward and reverse considered)

In [173]:
#read in sequence
seq=open("rosalind_orfr.txt").read().replace("\n", "")

#initialize dictionary for adding ORFs
lengths={}

#for every ORF, translate. Add to dictionary if aa sequence starts with "M"
for nt in range(0,len(seq)):
    orf=Seq(seq[nt:])
    translated=translate(orf, to_stop=True,table=1)
    aaStr=str(translated)
    if len(aaStr)>0:
        if aaStr[0]=="M":
            lengths[len(aaStr)]=aaStr


#take reverse complement of input sequence before looping through all ORFs and translating
rc=str(Seq(seq).reverse_complement())
for nt in range(0,len(rc)):
    orf=Seq(rc[nt:])
    translated=translate(orf, to_stop=True,table=1)
    aaStr=str(translated)
    if len(aaStr)>0:
        if aaStr[0]=="M":
            lengths[len(aaStr)]=aaStr
    

print(lengths[max(lengths)] + " is the longest ORF")


MTVPRSRPGPALGGVNIPSWTRGTYTEYVCWKVPYNFDTQTGVSTIGEKGPI is the longest ORF
